# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770912456936                   -0.52    9.0         
  2   -2.772149486566       -2.91       -1.32    1.0    139ms
  3   -2.772170270585       -4.68       -2.45    1.0    157ms
  4   -2.772170650681       -6.42       -3.18    1.0    142ms
  5   -2.772170722821       -7.14       -4.29    2.0    175ms
  6   -2.772170722864      -10.37       -4.39    1.0    172ms
  7   -2.772170723011       -9.83       -5.21    1.0    147ms
  8   -2.772170723013      -11.61       -5.40    1.0    160ms
  9   -2.772170723015      -11.81       -6.29    1.0    159ms
 10   -2.772170723015      -13.03       -6.66    2.0    177ms
 11   -2.772170723015      -13.72       -6.93    1.0    172ms
 12   -2.772170723015   +  -13.92       -7.59    1.0    154ms
 13   -2.772170723015      -14.27       -8.10    1.0    193ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ----

1.7735583436015256

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770719737804                   -0.53    9.0         
  2   -2.772048595784       -2.88       -1.31    1.0    139ms
  3   -2.772082519938       -4.47       -2.60    1.0    141ms
  4   -2.772083415916       -6.05       -4.11    2.0    195ms
  5   -2.772083417755       -8.74       -4.61    2.0    162ms
  6   -2.772083417809      -10.27       -5.85    1.0    146ms
  7   -2.772083417811      -11.80       -6.44    2.0    197ms
  8   -2.772083417811      -13.57       -6.89    2.0    172ms
  9   -2.772083417811   +  -13.64       -7.66    1.0    157ms
 10   -2.772083417811      -13.94       -9.03    2.0    178ms

Polarizability via ForwardDiff:       1.7725349720801444
Polarizability via finite difference: 1.7735583436015256
